# Purpose
The goal of this notebook is to make a master CSV. 



In [259]:
import pandas as pd
import re
import numpy as np
from dateutil import parser

## Functions

In [260]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \((.*)\)"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
def change_percent(test):
    if isinstance(test, str):
        query = r"%"
        m = re.search(query, test)

        if m:
            test = test.strip("%")
            test = float(test)/100
            return test
        else:
            return test
    else:
        return test
    
def get_state(test):
    query = r"([A-Z][A-Z])\Z"
    m = re.search(query, test)
    return m.group()


### Bringing in Pending 2014 Wait Times

In [261]:
# A function to format 
def make_df(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,24,25,26],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Pending_1412 = make_df("Pending","1412","Wait_Time/Pending_Clean/14_12_01_Wait_Pending.csv")
Pending_1412.head()

IOError: File Wait_Time/Pending_Clean/14_12_01_Wait_Pending.csv does not exist

# Staff Size

In [ ]:
staff = pd.read_csv("Staff_Size/Onboard_By_statyion_By_FY.csv")
staff = staff.drop(0).drop(1) # dropping first and second lines because these don't refer to actual stations.
staff["Station"] = staff["ORGANIZATION"].apply(get_station)
staff.head()
#staff

### Trying To Consolidate Staff And Pending

In [ ]:
#Grabbing all stations in pending that have duplicates
multiple_pending_series = Pending_1412["Station"].value_counts()
multiple_pending_dataframe = multiple_pending_series.to_frame()
multiple_pending_list = multiple_pending_dataframe[multiple_pending_dataframe["Station"] > 1].index.tolist()

pending_unique = pd.DataFrame()
for item in multiple_pending_list:
    new = Pending_1412[Pending_1412["Station"]==item]
    pending_unique = pending_unique.append(new)
    
    
#grabbing all in staff that aren't in pending...  
in_staff_not_pending = [x for x in staff["Station"].tolist() if x not in Pending_1412["Station"].tolist()]
staff_unique = pd.DataFrame()
for item in in_staff_not_pending:
    new = staff[staff["Station"]==item]
    staff_unique = staff_unique.append(new)
    
# ...and including that in a dataframe with the uniques from 
for item in multiple_pending_list:
    new = staff[staff["Station"]==item]
    staff_unique = staff_unique.append(new)



#### Cleaning
Identified areas that need to be clean and stations that match. At this point I'm going to try and generate unique IDs and then I'm going to bring in other data to join. 

Many of the locations in station in the hiring csv don't match the wait time csv. 


In [ ]:
# identified these as not stations, but facilities. Dropping from dataframe. 
to_drop = [18,29,50,59,66,77,87,96,105,114,123,144,152]
for index in to_drop:
    Pending_1412 = Pending_1412.drop(index)



### Generating Unique IDs

In [ ]:
IDs = []
query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"
#m = re.search(query, test)
#m.group(3)



for index, row in Pending_1412.iterrows():
    if row["Station"] not in IDs:
        m = re.search(query, row["Location"])
        loc = m.group(3).lower()
        IDs.append(row["Station"])

    else:
        m = re.search(query, row["Location"])
        loc = m.group(3).lower()
        IDs.append(row["Station"]+loc)
Pending_1412["ID"] = IDs
Pending_1412.head()

In [ ]:
#manually changing some IDs in staff based on shared location
staff["ID"] = staff["Station"]
staff["ID"][11] = "528"
staff["ID"][92] = "657st"
staff["ID"][12] = "528bu"
staff.head()

In [ ]:
master = Pending_1412.merge(staff, on="ID", how="left")
master["Station"] = master["Station_x"]
master= master.drop("Station_x",1).drop("Station_y",1)
print master.columns

#### So what was the result of our merge?

In [ ]:
def check_len(master):
    check = len(master) - 141
    if check == 0:
        print "Hurray! We didn't lose any rows."
    if check < 0:
        print "Uh oh. We lost {} rows!".format(str(abs(check)))
    if check > 0:
        print "we somehow...gained {} row? Well that's not right.".format(str(abs(check)))
check_len(master)

Great! Now I'm going to redefine ID so that it better matches with future dataframes.

In [ ]:
ID = []
query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"

for index, row in master.iterrows():
    m = re.search(query, row["Location"])
    loc = m.group(3).lower()
    ID.append(row["Station"]+loc)

master["ID"] = ID
master["ID"][72] = "537je" # edge case
master.head()



## Bringing in Pending 2016

In [ ]:
#functions to format 2016 
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,20,21,22],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Pending_1610 = make_df_2016("Pending","1610","Wait_Time/Pending_Clean/16_10_01_Wait_Pending.csv")
#Pending_1610 = Pending_1610.drop(297).drop(369) # edge case causing issues, not a station anyway

In [ ]:
def generate_IDs(df):
    ID = []
    for index, row in df.iterrows():
        #print index
        if len(row["Station"]) > 3:
            #print row["Station"]
            ID.append(None)
        else:
            try:
                #print index
                #print "entering else"
                #print "This is the location: "+row["Location"]
                query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"
                m = re.search(query, row["Location"].upper())
                new_ID = row["Station"]+m.group(3).lower()
                if new_ID not in ID:
                    ID.append(new_ID)
                else:
                    ID.append(None)
                #print m.group(3).upper()
            except AttributeError:
                #print "AttributeError"
                ID.append(None)
    return ID
ID = generate_IDs(Pending_1610)
Pending_1610["ID"] = ID

In [ ]:
Pending_1610 = Pending_1610.dropna(subset = ["ID"])
master = master.merge(Pending_1610,on="ID")
print master.columns
master["Station"] = master["Station_x"]
master["Location"] = master["Location_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)
master.head()

In [ ]:
check_len(master)

## Bringing in Complete 2016

In [ ]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,13,14,15],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Complete_1608 = make_df_2016("Complete","1608","Wait_Time/Completed_Cleaned/16_08_31_Wait_Complete.csv")
Complete_1608.head()

In [ ]:
len(Complete_1608)

In [ ]:
ID = generate_IDs(Complete_1608)
Complete_1608["ID"] = ID
Complete_1608 = Complete_1608.dropna(subset = ["ID"])
master = master.merge(Complete_1608,on="ID")
master["Station"] = master["Station_x"]
master["Location"] = master["Location_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)

In [ ]:
master.columns

In [ ]:
check_len(master)

In [ ]:
# Reference for duplicates I need to check out. 
master[master.duplicated(subset="Station", keep=False)][["Location","Station","ID"]]

# Bring in November Complete 2016

In [ ]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,13,14,15],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv

# Complete_1611 = make_df_2016("Complete","1611","Wait_Time/Completed_Cleaned/16_11_Wait.csv")

# ID = generate_IDs(Complete_1611)
# Complete_1611["ID"] = ID
# Complete_1611 = Complete_1611.dropna(subset = ["ID"])
# master = master.merge(Complete_1611,on="ID",how='left')
# master["Station"] = master["Station_x"]
# master["Location"] = master["Location_x"]
# master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)


# Bring In November Pending 2016

## Bring in Hiring

I'm going to have an issue dealing with the stations with the same ID. 

I made a new CSV called Hiring_Clean where I put the appropriate ID in a new ID column. I put ignore for those that were impossible to join accurately. 

In [ ]:
master.columns

In [ ]:
#Hiring = pd.read_csv("Hiring/Hiring_Clean.csv")
#Hiring = Hiring[Hiring["ID"]!= "Ignore"] 
#Hiring_summed = pd.pivot_table(Hiring,index=["Station"],values=["NbrEmps"],aggfunc=np.sum).reset_index()
#Hiring_summed = Hiring_summed.rename(columns = {"NbrEmps":"Choice_Hires"})

In [ ]:
#Hiring = pd.read_csv("Hiring/Hiring_Clean.csv")
Hiring = pd.read_csv("Hiring/Hiring_Clean_All_Positions.csv")
for_dups = []
for index, row in Hiring.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
Hiring["for_dups"] = for_dups
Hiring_summed = pd.pivot_table(Hiring,index=["for_dups"],values=["NbrEmps"],aggfunc=np.sum).reset_index()
Hiring_summed = Hiring_summed.rename(columns = {"NbrEmps":"Choice_Hires"})


dup_list = master[master.duplicated(subset="Station",keep=False)]["ID"].tolist()
dup_list.append("612n.")
dup_list.append("537je")

for_dups = []
for index, row in master.iterrows():
    if row["ID"] in dup_list:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
master["for_dups"] = for_dups

In [ ]:
master.columns

In [ ]:
master = master.merge(Hiring_summed, on="for_dups", how="left")
master.head()

In [ ]:
check_len(master)

## Bring in Leadership

Also creating a version of the CSV to deal with duplicate stations.

In [ ]:
Leadership = pd.read_csv("Leadership/Leadership_cleaned.csv")
Leadership = Leadership[Leadership["ID"] != "Ignore"]
Leadership["Vacancy"] = Leadership["Acting/Detailed"] == "VACANT"
Leadership["Interim"] = Leadership["Acting/Detailed"] != "VACANT"


# For easy merging


for_dups = []
for index, row in Leadership.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
Leadership["for_dups"] = for_dups

Leadership_summed = pd.pivot_table(Leadership, index=["for_dups"],values=["Vacancy","Interim"],aggfunc=np.sum).reset_index()
#Leadership_summed = Leadership_summed.merge(Leadership[["Station","ID"]], on="Station")












In [ ]:
master = master.merge(Leadership_summed, on="for_dups", how="left")
master["Interim"] = master["Interim"].fillna(0)
master["Vacancy"] = master["Vacancy"].fillna(0)

In [ ]:
check_len(master)

In [ ]:
master.head()

In [ ]:
master.columns

## Bringing in Completed 2014

In [ ]:
pd.read_csv("Wait_Time/Completed_Cleaned/14_09_30_Wait_Complete.csv").head()

In [ ]:
def make_df_complete_2014(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,12,13,14],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Complete_1409 = make_df_complete_2014("Complete","1409","Wait_Time/Completed_Cleaned/14_09_30_Wait_Complete.csv")
Complete_1409["ID"] = generate_IDs(Complete_1409)
Complete_1409

In [ ]:
for_dups = []
for index, row in Complete_1409.iterrows():
    if row["ID"] in dup_list:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
Complete_1409["for_dups"] = for_dups

In [ ]:
columns_to_use = Complete_1409.columns.difference(master.columns).tolist()
columns_to_use.append("for_dups")

master = master.merge(Complete_1409[columns_to_use], on="for_dups", how="left")
master.columns






In [ ]:
master.head()

In [ ]:
check_len(master)    

# Turnover

In [ ]:
turnover = pd.read_csv("Turnover/Turnover_Physician.csv",na_values="-")
# to drop national numbers and visn
turnover = turnover[turnover["Organization"].str.len() > 5]

turnover["Station"]= turnover["Organization"].apply(get_station)

for_dups = []
for index, row in turnover.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
turnover["for_dups"] = for_dups


turnover.head()

### Not merging in all of turnover

To do this would create a massive number of columns. Instead, it makes more sense to merge filtered versions in on a case by case basis.

But...it does make sense to merge in specialty 

In [ ]:
turnover.to_csv("turnover/Turnover_Physician_For_Merging.csv")

In [ ]:

physician_turnover = pd.read_csv("Turnover/Turnover_Physician_For_Merging.csv",na_values="-",
                           usecols=[2,3,4,5,6,7,10],
                           skiprows=[0], #skipping the original header
                           names=["Specialty","Physician_Turnover_FY11","Physician_Turnover_FY12",
                                  "Physician_Turnover_FY13","Physician_Turnover_FY14","Physician_Turnover_FY15",
                                  "for_dups"])
physician_turnover = physician_turnover[physician_turnover["Specialty"]=="0602 Physician (All Specialties)"].drop("Specialty",1)
master = master.merge(physician_turnover,on="for_dups",how="left")

# Looking at pyschiatrists

In [ ]:
psychiatrists_hiring = pd.read_csv("Hiring/Hiring_Psychiatrist.csv")
psychiatrists_hiring
for_dups = []
for index, row in psychiatrists_hiring.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
psychiatrists_hiring["for_dups"] = for_dups
psychiatrists_hiring["NbrPsychiatrists"] = psychiatrists_hiring["NbrEmps"]

master = master.merge(psychiatrists_hiring[["NbrPsychiatrists","for_dups"]], on="for_dups", how="left")


# Fixing Percents

In [ ]:
master = master.applymap(change_percent)

# Self Generated Columns

In [ ]:
# Appts
master["Pending_Increase"] = master["Appts_Pending_1610"] - master["Appts_Pending_1412"]
master["Pending_Increase_%"] = (master["Pending_Increase"]/master["Appts_Pending_1412"])*100
master["Complete_Increase"] = master["Appts_Complete_1608"] - master["Appts_Complete_1409"]
master["Complete_Increase_%"] = (master["Complete_Increase"]/master["Appts_Complete_1409"])*100
master["Complete_Pending_Appts_Diff_14"] = master["Appts_Pending_1412"] - master["Appts_Complete_1409"]
master["Complete_Pending_Appts_Diff_16"] = master["Appts_Pending_1610"] - master["Appts_Complete_1608"]

master["Complete_Pending_Appts_Diff_14_%"] = ((master["Appts_Pending_1412"] - master["Appts_Complete_1409"])/master["Appts_Complete_1409"])*100
master["Complete_Pending_Appts_Diff_16_%"] = ((master["Appts_Pending_1610"] - master["Appts_Complete_1608"])/master["Appts_Complete_1608"])*100




master["Complete_Pending_Gap_Increase_%"] = (master["Complete_Pending_Appts_Diff_16"] -
                                             master["Complete_Pending_Appts_Diff_14"]) / master["Complete_Pending_Appts_Diff_14"]

# Leadership
master["Missing_Leadership"] = master["Vacancy"] + master["Interim"]

#Staff
master["Staff_11_To_12_Increase"] = master["MAY-FY12"] - master["MAY-FY11"]
master["Staff_11_To_12_Increase_%"] = (master["Staff_11_To_12_Increase"]/master["MAY-FY12"])*100

master["Staff_12_To_13_Increase"] = master["MAY-FY13"] - master["MAY-FY12"]
master["Staff_12_To_13_Increase_%"] = (master["Staff_12_To_13_Increase"]/master["MAY-FY13"])*100

master["Staff_13_To_14_Increase"] = master["MAY-FY14"] - master["MAY-FY13"]
master["Staff_13_To_14_Increase_%"] = (master["Staff_13_To_14_Increase"]/master["MAY-FY14"])*100

master["Staff_14_To_15_Increase"] = master["MAY-FY15"] - master["MAY-FY14"]
master["Staff_14_To_15_Increase_%"] = (master["Staff_14_To_15_Increase"]/master["MAY-FY15"])*100

master["Staff_15_To_16_Increase"] = master["MAY-FY16"] - master["MAY-FY15"]
master["Staff_15_To_16_Increase_%"] = (master["Staff_15_To_16_Increase"]/master["MAY-FY16"])*100

master["Staff_14_To_16_Increase"] = master["MAY-FY16"] - master["MAY-FY14"]
master["Staff_14_To_16_Increase_%"] = (master["Staff_14_To_16_Increase"]/master["MAY-FY16"])*100

# Hires
master["Choice_Increase_%"] = master["Choice_Hires"]/master["MAY-FY14"]
master["Choice_Staff_Dif"] = master["Staff_14_To_16_Increase"] - master["Choice_Hires"] 
master["Choice_Increase_%_PC_Wait_Complete_1409_Ratio"] = master["PC_Wait_Complete_1409"]/master["Choice_Increase_%"]
master["Choice_Increase_%_PC_Wait_Pending_1412_Ratio"] = master["PC_Wait_Pending_1412"]/master["Choice_Increase_%"]

# Wait Times
master["PC_Wait_Pending_Increase"] = master["PC_Wait_Pending_1610"] - master["PC_Wait_Pending_1412"]
master["PC_Wait_Pending_Increase_%"] = (master["PC_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["PC_Wait_Complete_Increase"] = master["PC_Wait_Complete_1608"] - master["PC_Wait_Complete_1409"]
master["PC_Wait_Complete_Increase_%"] = (master["PC_Wait_Complete_Increase"]/master["PC_Wait_Complete_1409"])*100

master["SC_Wait_Pending_Increase"] = master["SC_Wait_Pending_1610"] - master["SC_Wait_Pending_1412"]
master["SC_Wait_Pending_Increase_%"] = (master["SC_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["SC_Wait_Complete_Increase"] = master["SC_Wait_Complete_1608"] - master["SC_Wait_Complete_1409"]
master["SC_Wait_Complete_Increase_%"] = (master["SC_Wait_Complete_Increase"]/master["SC_Wait_Complete_1409"])*100

master["MH_Wait_Pending_Increase"] = master["MH_Wait_Pending_1610"] - master["MH_Wait_Pending_1412"]
master["MH_Wait_Pending_Increase_%"] = (master["MH_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["MH_Wait_Complete_Increase"] = (master["MH_Wait_Complete_1608"] - master["MH_Wait_Complete_1409"])*100
master["MH_Wait_Complete_Increase_%"] = (master["MH_Wait_Complete_Increase"]/master["MH_Wait_Complete_1409"])*100

# Staff size relative to appointments

master["Staff_To_Appts_Pending_14"] = master["MAY-FY14"]/master["Appts_Pending_1412"]
master["Staff_To_Appts_Pending_16"] = master["MAY-FY16"]/master["Appts_Pending_1610"]
master["Staff_To_Appts_Complete_14"] = master["MAY-FY14"]/master["Appts_Complete_1409"]
master["Staff_To_Appts_Complete_16"] = master["MAY-FY16"]/master["Appts_Complete_1608"]


# Appointments Over 30

master["Percentage_Point_Wait_Over_30_Increase_Pending"] = master['%_Appts_Over_30_Pending_1610'] - master['%_Appts_Over_30_Pending_1412']
master["Percentage_Point_Wait_Over_30_Increase_Complete"] = master['%_Appts_Over_30_Complete_1608'] - master['%_Appts_Over_30_Complete_1409']

# Get State

master["State"] = master["Location"].apply(get_state)



# Export to CSV

In [ ]:
master.to_csv("Master/Master.csv", index=False)

In [ ]:
master.columns

In [ ]:
for column in master.columns:
    print column

# National

In [ ]:
#Taken from VA site
national_complete = pd.read_csv("Wait_Time/Completed_Cleaned/National_Complete.csv")
national_pending = pd.read_csv("Wait_Time/Pending_Clean/National_Pending.csv")


#used to strip time from Pending_Time
def get_date(test):
    test = test.strip('National: ')
    return parser.parse(test)

national_complete["date"] = national_complete["Complete_Time"].apply(get_date)
national_pending["date"] = national_pending["Pending_Time"].apply(get_date)
national_complete.sort("date")
national_pending.sort("date")

national_pending = national_pending.drop(20) #for some reason this was read in as 1989. Just dropping it

#national_pending.to_csv("National/National_Pending.csv",index=False)
#national_complete.to_csv("National/National_Complete.csv",index=False)

#NC = national_complete[["Appts_Complete_National","date"]]
#NP = national_pending[["Appts_Pending_National","date"]]